
# **Asignatura de Big Data Analytics**

Prof. José M. Luna

jmluna@uco.es

---

En primer lugar, debemos instalar todas las dependencias necesarias para poder ejecutar el algoritmo. Así, tenemos que instalar Java 8, Spark 3.0 con Hadoop 3.2, y findspark para alojar spark en el sistema (acceder a Spark desde un Notebook).

In [ ]:
# Instalacion de java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Instalacion de spark
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# Descomprimimos el fichero de spark
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# Instalamos findspark
!pip install -q findspark

# Establecemos las variables de entorno para Java y Spark 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

Ejecutamos una sesion local de spark para comprobar que esta funcionando.

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Importamos el conjunto de datos con el que queremos trabajar y comprobamos que se ha subido correctamente

In [ ]:
from google.colab import files
uploaded = files.upload()

!ls

Saving BostonHousing.csv to BostonHousing.csv
BostonHousing.csv  spark-3.0.0-bin-hadoop3.2
sample_data	   spark-3.0.0-bin-hadoop3.2.tgz


Leemos el dataset con el que queremos trabajar, en este caso vamos a trabajar con el dataset de BostonHousing



In [ ]:
dataset = spark.read.csv('BostonHousing.csv',inferSchema=True, header =True)

Mostramos los atributos que tiene el dataset y su tipo

In [ ]:
dataset.printSchema()

root
 |-- crim: double (nullable = true)
 |-- zn: double (nullable = true)
 |-- indus: double (nullable = true)
 |-- chas: integer (nullable = true)
 |-- nox: double (nullable = true)
 |-- rm: double (nullable = true)
 |-- age: double (nullable = true)
 |-- dis: double (nullable = true)
 |-- rad: integer (nullable = true)
 |-- tax: integer (nullable = true)
 |-- ptratio: double (nullable = true)
 |-- b: double (nullable = true)
 |-- lstat: double (nullable = true)
 |-- medv: double (nullable = true)



En primer lugar, vamos a trabajar con una sola variable y una salida. Vamos a coger, por ejemplo, la variable crim y la varible medv (salida)

In [ ]:
subdataset = dataset.select("crim","medv")
subdataset.printSchema()
subdataset.show()

root
 |-- crim: double (nullable = true)
 |-- medv: double (nullable = true)

+-------+----+
|   crim|medv|
+-------+----+
|0.00632|24.0|
|0.02731|21.6|
|0.02729|34.7|
|0.03237|33.4|
|0.06905|36.2|
|0.02985|28.7|
|0.08829|22.9|
|0.14455|27.1|
|0.21124|16.5|
|0.17004|18.9|
|0.22489|15.0|
|0.11747|18.9|
|0.09378|21.7|
|0.62976|20.4|
|0.63796|18.2|
|0.62739|19.9|
|1.05393|23.1|
| 0.7842|17.5|
|0.80271|20.2|
| 0.7258|18.2|
+-------+----+
only showing top 20 rows



Creamos un nuevo conjunto de datos donde los inputs serán un vector (necesario para LinearRegression), pero consideramos sólo un atributo

In [ ]:
from pyspark.ml.feature import VectorAssembler

df = VectorAssembler(inputCols=['crim'], outputCol='inputs').transform(subdataset)

df = df.select(['inputs','medv'])

df.printSchema()

root
 |-- inputs: vector (nullable = true)
 |-- medv: double (nullable = true)



Dividimos el conjunto de datos en train y test. Vamos a considerar una partición del tipo 80/20

In [ ]:
trainData,testData = df.randomSplit([0.8,0.2])

Indicamos qué atributos serán las entradas y cuales serán la salida. En este caso, vamos a tomar como entrada "crim" y como salida "medv". Finalmente, entrenamos y testeamos con trainData y testData, y mostramos las predicciones

In [ ]:
from pyspark.ml.regression import LinearRegression

regressor = LinearRegression(featuresCol = 'inputs', labelCol = 'medv')

#Entrenamos el modelo con el conjunto de train
regressor = regressor.fit(trainData)

#Evaluamos el modelo con el conjunto de test
pred = regressor.evaluate(testData)

#Mostramos las predicciones
pred.predictions.show()

+---------+----+------------------+
|   inputs|medv|        prediction|
+---------+----+------------------+
|[0.01501]|24.5|23.861244119853577|
|[0.01538]|44.0| 23.86110047364677|
|[0.02177]|42.3| 23.85861966483186|
|[0.02498]|16.5| 23.85737343692954|
|[0.02731]|21.6|23.856468854059624|
|[0.03306]|20.6|  23.8542365143592|
|[0.03445]|24.1|23.853696870501185|
|[0.03551]|22.9|23.853285343530327|
|[0.03659]|24.8| 23.85286605189964|
|[0.03768]|34.6|23.852442877939033|
|[0.03961]|21.1|23.851693588265675|
| [0.0459]|22.3|23.849251602749906|
|[0.04819]|21.9|23.848362549199653|
|[0.05425]|24.6|  23.8460098572719|
|[0.05644]|32.4| 23.84515962702078|
| [0.0566]|23.6| 23.84509750974216|
|[0.06263]|22.4| 23.84275646480415|
|[0.06588]|39.8| 23.84149470758217|
|[0.06617]|19.3| 23.84138212001467|
|[0.06642]|29.9| 23.84128506176683|
+---------+----+------------------+
only showing top 20 rows



Mostramos resultados del modelo de regresión en train

In [ ]:
#coeficientes del modelo de regresion
coeff = regressor.coefficients
#interseccioon del model con Y
intr = regressor.intercept
print ("Coeficientes: %a" %coeff)
print ("Interseccion: %f" %intr)

#Entrenamiento
print("Root Mean Squared Err (RMSE): %f" % regressor.summary.rootMeanSquaredError)
print("Mean Absolute Err (MAE): %f" % regressor.summary.meanAbsoluteError)
print("Mean Squared Err (MSE): %f" % regressor.summary.meanSquaredError)

Coeficientes: DenseVector([-0.3882])
Interseccion: 23.867071
Root Mean Squared Err (RMSE): 8.413242
Mean Absolute Err (MAE): 6.121929
Mean Squared Err (MSE): 70.782645


Mostramos resultados del modelo de regresión en test

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

rmse = RegressionEvaluator(labelCol="medv", predictionCol="prediction", metricName="rmse").evaluate(pred.predictions)
print ("Root Mean Squared Err (RMSE): %f" % rmse)
mae = RegressionEvaluator(labelCol="medv", predictionCol="prediction", metricName="mae").evaluate(pred.predictions)
print ("Mean Absolute Err (MAE): %f" % mae)
mse = RegressionEvaluator(labelCol="medv", predictionCol="prediction", metricName="mse").evaluate(pred.predictions)
print ("Mean Squared Err (MSE): %f" % mse)

Root Mean Squared Err (RMSE): 8.230148
Mean Absolute Err (MAE): 6.420361
Mean Squared Err (MSE): 67.735335
